In [3]:
ALTER PROCEDURE GetDriverStatistics
    @DriverId INT
AS
BEGIN
    -- Selecting driver details, total trips, average feedback, and latest trip details
    SELECT 
        D.DriverID,
        U.UserFName + ' ' + U.UserLName AS DriverName,
        COUNT(RR.RequestId) AS TotalTrips,
        AVG(F.Rating) AS AverageFeedbackRating, --Commenting out currently. Maybe add a rating column in feedback
        MAX(RR.ReqDateTime) AS LatestTripDateTime,
        MAX(RR.DestinationLocation) AS LatestTripDestination
    FROM Driver D
        -- Joining with the User table to get driver's name
        JOIN [User] U ON D.UserID = U.UserID
        -- Left joining with RideRequest to include drivers with no trips
        LEFT JOIN RideRequest RR ON D.DriverID = RR.VehicleID
        -- Left joining with Feedback to include drivers with no feedback
        LEFT JOIN Feedback F ON D.DriverID = F.DriverID
    WHERE D.DriverID = @DriverId
    -- Grouping results by driver for aggregate functions
    GROUP BY D.DriverID, U.UserFName, U.UserLName;
END;




Commands completed successfully.

Total execution time: 00:00:00.007

In [4]:
EXEC GetDriverStatistics 1

(1 row affected)

Total execution time: 00:00:00.005

DriverID,DriverName,TotalTrips,AverageFeedbackRating,LatestTripDateTime,LatestTripDestination
1,Alice Johnson,2,5,2023-11-26 11:00:00.000,202 Walnut St


In [5]:
CREATE PROCEDURE GetUpcomingServiceRequests
AS
BEGIN
    -- Selecting details for upcoming service requests
    SELECT 
        SR.SerReqId,
        V.VehicleType,
        U.UserFName + ' ' + U.UserLName AS CustomerName,
        SR.ReqDateTime AS RequestDateTime,
        SR.ServiceDueDate AS DueDate,
        SR.PreviousServiceDate AS LastServiceDate
    FROM ServiceRequest SR
        -- Joining with Vehicle table to get vehicle details
        JOIN Vehicle V ON SR.VehicleId = V.VehicleId
        -- Joining with User table to get customer details
        JOIN [User] U ON V.VehicleId = U.UserID
    WHERE SR.ServiceDueDate > GETDATE();  -- Filtering only upcoming service requests
END;




Commands completed successfully.

Total execution time: 00:00:00.005

In [6]:
EXEC GetUpcomingServiceRequests

(5 rows affected)

Total execution time: 00:00:00.012

SerReqId,VehicleType,CustomerName,RequestDateTime,DueDate,LastServiceDate
1,SUV,Alice Johnson,2023-11-24 10:00:00.000,2023-12-01,2023-11-01
2,Compact,Eva Williams,2023-11-25 16:00:00.000,2023-12-05,2023-11-01
3,Sedan,John Doe,2023-11-26 12:00:00.000,2023-12-10,2023-11-01
4,Truck,Bob Smith,2023-11-27 20:00:00.000,2023-12-15,2023-11-01
5,Convertible,Chris Taylor,2023-11-29 00:00:00.000,2023-12-20,2023-11-01


In [7]:
CREATE PROCEDURE spCalculateCustomerLoyaltyDiscount 
    @CustomerId INT
AS
BEGIN
    DECLARE @NumRides INT
    DECLARE @DiscountPercent INT 
    
    SELECT @NumRides = COUNT(*)
    FROM RideRequest r
    INNER JOIN Customer c ON r.CustomerID = c.CustomerID
    WHERE c.CustomerID = @CustomerId
        AND r.ReqDateTime BETWEEN DATEADD(YEAR, -1, GETDATE()) AND GETDATE()
            
    SET @DiscountPercent = 
        CASE 
            WHEN @NumRides > 10 THEN 20
            WHEN @NumRides BETWEEN 5 AND 10 THEN 10
            ELSE 0
        END
        
    SELECT @CustomerId AS CustomerId, @DiscountPercent AS LoyaltyDiscountPercentage  
END

Commands completed successfully.

Total execution time: 00:00:00.005

In [8]:
EXEC spCalculateCustomerLoyaltyDiscount 1

(1 row affected)

Total execution time: 00:00:00.011

CustomerId,LoyaltyDiscountPercentage
1,0
